In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install optuna optuna-integration[tfkeras]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.9 MB/s eta 0:00:00


In [3]:
# Step 1: Import Libraries and Set Up Environment
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from transformers import RobertaTokenizer, TFRobertaModel

from sklearn.utils.class_weight import compute_class_weight

import optuna
from optuna.integration import TFKerasPruningCallback

from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

from sklearn.ensemble import RandomForestClassifier

import os
import random
from tqdm import tqdm  # Added tqdm for progress visualization
import h5py  # Added h5py for saving data
import gc

from concurrent.futures import ThreadPoolExecutor

# Initialize GPU settings if available
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "physical GPUs,", len(logical_gpus), "logical GPUs.")
    except RuntimeError as e:
        print(e)

1 physical GPUs, 1 logical GPUs.


In [5]:
!cp -r "/content/drive/MyDrive/CS5344 Project Data/max_length(token)256/Test Train Split/Each/"* /content

In [6]:
!cp "/content/drive/MyDrive/CS5344 Project Data/max_length(token)256/final_model.keras" /content

In [13]:
import tensorflow as tf
from transformers import TFAutoModel

# 使用 tf.keras.utils.register_keras_serializable
@tf.keras.utils.register_keras_serializable(package="Custom", name="TransformerLayer")
class TransformerLayer(tf.keras.layers.Layer):
    def __init__(self, model_name='roberta-base', **kwargs):
        super(TransformerLayer, self).__init__(**kwargs)
        self.transformer = TFAutoModel.from_pretrained(model_name)
        self.transformer.trainable = False  # 根据需要调整

    def call(self, inputs):
        input_ids, attention_mask = inputs
        outputs = self.transformer(input_ids, attention_mask=attention_mask)
        return outputs.last_hidden_state[:, 0, :]  # 提取 [CLS] token 的嵌入

    def get_config(self):
        config = super().get_config()
        config.update({
            'model_name': 'roberta-base'  # 或者是实际传递的模型名称
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)


In [14]:


# 加载已训练的 Keras 模型
final_model_path = 'final_model.keras'  # 替换为模型路径
final_model = load_model(final_model_path)

# 假设加载的模型包含 transformer 层
embedding_extractor = EmbeddingExtractor(final_model)

# 定义输入
input_ids_in = tf.keras.Input(shape=(256,), dtype=tf.int32, name='input_ids')
input_masks_in = tf.keras.Input(shape=(256,), dtype=tf.int32, name='attention_mask')

# 构建嵌入模型
embeddings = embedding_extractor([input_ids_in, input_masks_in])
embedding_model = tf.keras.Model(inputs=[input_ids_in, input_masks_in], outputs=embeddings)

# 打印模型结构确认
embedding_model.summary()

TypeError: <class 'keras.src.models.functional.Functional'> could not be deserialized properly. Please ensure that components that are Python object instances (layers, models, etc.) returned by `get_config()` are explicitly deserialized in the model's `from_config()` method.

config={'module': 'keras.src.models.functional', 'class_name': 'Functional', 'config': {'name': 'functional', 'trainable': True, 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_shape': [None, 256], 'dtype': 'int32', 'sparse': False, 'name': 'input_ids'}, 'registered_name': None, 'name': 'input_ids', 'inbound_nodes': []}, {'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_shape': [None, 256], 'dtype': 'int32', 'sparse': False, 'name': 'attention_mask'}, 'registered_name': None, 'name': 'attention_mask', 'inbound_nodes': []}, {'module': None, 'class_name': 'TransformerLayer', 'config': {'name': 'transformer_layer', 'trainable': True, 'dtype': {'module': 'kera

In [4]:
from transformers import TFAutoModel, RobertaTokenizer
model_name = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(model_name)
transformer_model = TFAutoModel.from_pretrained(model_name)
transformer_model.trainable = False

class EmbeddingExtractor(tf.keras.layers.Layer):
    def __init__(self, transformer_model, **kwargs):
        super(EmbeddingExtractor, self).__init__(**kwargs)
        self.transformer = transformer_model

    def call(self, inputs):
        input_ids, attention_mask = inputs
        outputs = self.transformer(input_ids, attention_mask=attention_mask)
        return outputs.last_hidden_state[:, 0, :]  # 提取 [CLS] token 的嵌入

input_ids_in = tf.keras.Input(shape=(256,), dtype=tf.int32, name='input_ids')
input_masks_in = tf.keras.Input(shape=(256,), dtype=tf.int32, name='attention_mask')

# 使用自定义的嵌入提取器
embedding_extractor = EmbeddingExtractor(transformer_model)
embeddings = embedding_extractor([input_ids_in, input_masks_in])
embedding_model = tf.keras.Model(inputs=[input_ids_in, input_masks_in], outputs=embeddings)


KeyboardInterrupt: 